In [2]:
import cv2
import os
import numpy as np
from glob import glob
import random
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [3]:
def seed_everything(seed: int = 24):
    random.seed(seed)
    np.random.seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True

seed_everything()

In [4]:
closed_path = '../dataset/realImg/eyes-only-22/closed'
opened_path =  '../dataset/realImg/eyes-only-22/opened'

In [5]:
def get_dataset_paths(data_dir): # '../dataset/realImg/eyes-only-22'
    label_list = []

    opened_dir = os.path.join(data_dir, "opened")
    closed_dir = os.path.join(data_dir, "closed")

    opened_img_paths = glob(os.path.join(opened_dir, '*.jpg'))
    opened_img_paths.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    closed_img_paths = glob(os.path.join(closed_dir, '*.jpg'))
    closed_img_paths.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # "opened" 레이블은 1,  "closed" 레이블은 0으로 지정
    label_list.extend([1] * len(opened_img_paths))
    label_list.extend([0] * len(closed_img_paths))
    img_path_list = opened_img_paths + closed_img_paths
    return img_path_list, label_list


In [6]:
from sklearn.utils import shuffle

def shuffle_dataset(img_path_list, label_list):
    img_path_list, label_list = shuffle(img_path_list, label_list, random_state=42)
    return img_path_list, label_list

In [7]:
img_path_list, label_list = get_dataset_paths('../dataset/realImg/eyes-only-22')
all_img_path, all_label = shuffle_dataset(img_path_list, label_list)
print(all_label)

[1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __len__(self):
        return len(self.img_path_list)

    def __getitem__(self, index):  
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
    
        if self.transforms is not None:
            image = self.transforms(image)
            
        if self.train_mode:
            label = self.label_list[index]
            return image, label
    
        else:
            return image

In [9]:
# train : test: val = 0.8 : 0.1 : 0.1

train_len = int(len(all_img_path)*0.8)
test_len = int(len(all_img_path)*0.1)
val_len = int(len(all_img_path)*0.1)
              
train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

test_img_path = all_img_path[train_len: train_len + test_len]
test_label = all_img_path[train_len: train_len + test_len]

val_img_path = all_img_path[train_len + test_len:]
val_label = all_label[train_len + test_len:]


In [10]:
print('train, test, val length:', train_len, test_len, val_len)

train, test, val length: 4463 557 557


In [20]:
# 하이퍼파라미터 
CFG = {
    'IMG_SIZE':128,
    'EPOCHS':10, 
    'LEARNING_RATE':2e-2, 
    'BATCH_SIZE':12, 
    'SEED':24, 
}

In [12]:
train_transform = transforms.Compose([
    transforms.ToPILImage(), 
    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) 

])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [13]:
# train_batches = len(train_loader)
# test_batches = len(test_loader)
# 
# print('total train imgs :', train_len,'/ total train batches :', train_batches)
# print('total test imgs :', val_len, '/ total test batches :', test_batches)

In [14]:
# 사진 확인하기
# train_features, train_labels = next(iter(train_loader))  
# img = train_features[0]
# label = train_labels[0]
# plt.imshow(img[0], cmap="gray")
# plt.show()
# print(f"Label: {label}")

In [15]:
class Model(nn.Module):
    def __init__(self,):
        super().__init__()
        self.model = torchvision.models.resnet18(pretrained = False)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(1000, 10)

    def forward(self, images):
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [22]:
# k-fold
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 4)
folds=[]
for trn_idx,val_idx in skf.split(train_img_path, train_label):
    folds.append((trn_idx,val_idx))

In [21]:
num_epochs = CFG['EPOCHS'] 
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장

for i,fold in enumerate(range(4)):
    print('===============',i+1,'fold start===============')

    model = Model()
    optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'] )
    criterion = nn.CrossEntropyLoss() 
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size = 5,
                                                   gamma = 0.9) # learning rate scheduler 로 학습률 주기적 감소

    train_idx = folds[fold][0] 
    val_idx = folds[fold][1]


    train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    val_dataset = CustomDataset(val_img_path, val_label, train_mode=True, transforms=test_transform)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


    val_acc_max = 0.85
    val_loss_min = 0.3
    
    for epoch in range(num_epochs):
        #모델 학습
        for i, (images, targets) in enumerate(train_loader):
            model.train()

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            if (i+1) % 20 == 0:
                print(f'Epoch: {epoch} - Loss: {loss:.6f}')


        val_loss_list = []
        val_acc_list = []
        # 모델 검증
        for i, (images, targets) in enumerate(val_loader):
            model.eval()

            with torch.no_grad():
                outputs = model(images)
                val_loss = criterion(outputs,targets)

                preds = torch.argmax(outputs, dim=1)
            
                batch_acc = (preds==targets).float().mean() # boolean 값의 평균

                val_loss_list.append(val_loss)
                val_acc_list.append(batch_acc)

        val_loss = np.mean(val_loss_list)
        val_acc = np.mean(val_acc_list)

        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')

        # 정확도 기준 성능 좋은 모델 / 혹은 loss 기준 가능
        '''
         if valid_acc_max < val_acc:
            valid_acc_max = val_acc
            best_models.append(model)
            print('model save, model val acc : ',val_acc)
            print('best_models size : ',len(best_models))
            '''


        if val_loss_min > val_loss:
            val_loss_min = val_loss
            best_models.append(model)

    # lr 조절
    lr_scheduler.step()

=============== 1 fold start===============


C:\Users\Administrator\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch: 0 - Loss: 191.763306
Epoch: 0 - Loss: 6.679359
Epoch: 0 - Loss: 0.653717
Epoch: 0 - Loss: 2.368544
Epoch: 0 - Loss: 0.340592
Epoch: 0 - Loss: 0.190015
Epoch: 0 - Loss: 0.346462
Epoch: 0 - Loss: 0.861421
Epoch: 0 - Loss: 0.312239
Epoch: 0 - Loss: 0.341255
Epoch: 0 - Loss: 0.389755
Epoch: 0 - Loss: 0.292173
Epoch: 0 - Loss: 0.385165
Epoch: 0 - Loss: 0.242726
Epoch: 0 - Loss: 0.182806
Epoch: 0 - Loss: 0.197053
Epoch: 0 - Loss: 0.184065
Epoch: 0 - Loss: 0.936939
Epoch: 0 - valid Loss: 0.369604 - valid_acc : 0.856890
Epoch: 1 - Loss: 0.319737
Epoch: 1 - Loss: 0.117405
Epoch: 1 - Loss: 0.160632
Epoch: 1 - Loss: 0.070710
Epoch: 1 - Loss: 0.545494
Epoch: 1 - Loss: 0.173050
Epoch: 1 - Loss: 0.123139
Epoch: 1 - Loss: 0.210491
Epoch: 1 - Loss: 0.086899
Epoch: 1 - Loss: 0.051603
Epoch: 1 - Loss: 0.899853
Epoch: 1 - Loss: 0.136711
Epoch: 1 - Loss: 0.159089
Epoch: 1 - Loss: 0.051506
Epoch: 1 - Loss: 0.019256
Epoch: 1 - Loss: 0.031649
Epoch: 1 - Loss: 0.704862
Epoch: 1 - Loss: 0.208332
Epoch: 

In [42]:
# 모델 선택: 여기서는 best_models 리스트의 첫 번째 모델을 사용합니다.
model = best_models[0]

# 추론할 이미지 선택: 여기서는 첫 번째 테스트 이미지를 선택합니다.
image_index = 0  # 추론할 이미지의 인덱스
image, true_label = test_dataset[image_index]

# 모델 추론
model.eval()
with torch.no_grad():
    output = model(image.unsqueeze(0))  # 배치 차원 추가
    predicted_label = torch.argmax(output, dim=1).item()

# 클래스 레이블 가져오기 (예: 클래스 레이블이 문자열인 경우)
class_labels = ['class_0', 'class_1', 'class_2', ...]  # 클래스 레이블 리스트
predicted_class = class_labels[predicted_label]
true_class = class_labels[true_label.item()]

# 이미지와 추론 결과를 시각화
plt.figure()
plt.title(f'True Label: {true_class}, Predicted Label: {predicted_class}')
image = image.permute(1, 2, 0)  # 이미지 차원 순서 변경
plt.imshow(image)
plt.show()


폴드 1용 모델이 ../saved_models\model_fold_1.pth에 저장
폴드 2용 모델이 ../saved_models\model_fold_2.pth에 저장
폴드 3용 모델이 ../saved_models\model_fold_3.pth에 저장
폴드 4용 모델이 ../saved_models\model_fold_4.pth에 저장
폴드 5용 모델이 ../saved_models\model_fold_5.pth에 저장
폴드 6용 모델이 ../saved_models\model_fold_6.pth에 저장
폴드 7용 모델이 ../saved_models\model_fold_7.pth에 저장
폴드 8용 모델이 ../saved_models\model_fold_8.pth에 저장
폴드 9용 모델이 ../saved_models\model_fold_9.pth에 저장
폴드 10용 모델이 ../saved_models\model_fold_10.pth에 저장
폴드 11용 모델이 ../saved_models\model_fold_11.pth에 저장
폴드 12용 모델이 ../saved_models\model_fold_12.pth에 저장
폴드 13용 모델이 ../saved_models\model_fold_13.pth에 저장
폴드 14용 모델이 ../saved_models\model_fold_14.pth에 저장
폴드 15용 모델이 ../saved_models\model_fold_15.pth에 저장
폴드 16용 모델이 ../saved_models\model_fold_16.pth에 저장
폴드 17용 모델이 ../saved_models\model_fold_17.pth에 저장
폴드 18용 모델이 ../saved_models\model_fold_18.pth에 저장
폴드 19용 모델이 ../saved_models\model_fold_19.pth에 저장
폴드 20용 모델이 ../saved_models\model_fold_20.pth에 저장
폴드 21용 모델이 ../saved_models\model_fold_

In [24]:
test_dataset = CustomDataset(test_img_path, test_label, train_mode=False, transforms=None) 
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

preds = []
for idx,model in enumerate(best_models[:1]):
    print(idx+1, '번째 모델 예측 진행중')
    model = model
    model.eval()
    y_pred = []
    with torch.no_grad():
        for i, d in enumerate(test_loader):
            inputs = d.to('cuda')
            outputs = model(inputs).detach().cpu().numpy()
            y_pred.extend(outputs.argmax(axis=1).astype(int))

    preds.append(y_pred)

22


In [ ]:
model_folder = 'E:\\2023\\2023_1_1\comp\saved_models'
model_template = 'model_fold_{}.pth'

test_dataset = CustomDataset(test_img_path, test_label, train_mode=False, transforms=None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

preds = []
for model in range(1, 23):
    model_path = os.path.join(model_folder, model_template.format(model))
    if os.path.exists(model_path):
        print(f'Loading model {model}')

        model = torch.load(model_path)
        model.eval()

        y_pred = []
        with torch.no_grad():
            for i, d in enumerate(test_loader):
                inputs = d
                outputs = model(inputs).detach().cpu().numpy()
                y_pred.extend(outputs.argmax(axis=1).astype(int))
        preds.append(y_pred)
    else:
        print(f'Model {model} not found.')